# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 438000  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [14]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [16]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanCost($)"] + allAnnualData["HeatingCost($)"] + allAnnualData["CoolingCost($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData

In [17]:
# Export hourly data to csv, keeping units in kBtus to be able to combine heating, cooling and fans for baseline comparison
allAnnualData.to_csv("../Scraper_Output/hou_hourly_data.csv", header=True, index=True)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [18]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [19]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [20]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [21]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [22]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [23]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [24]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [25]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanCost($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingCost($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingCost($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [26]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [27]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner")

#allMonthlyData

In [28]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanCost($)"] + allMonthlyData["HeatingCost($)"] + allMonthlyData["CoolingCost($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [29]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

#allMonthlyData

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [30]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData2.sum()

In [31]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [32]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [33]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [34]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [35]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance.mean()

Illuminance(lux)
Scenario           Month                  
Clear/Clear         01         3732.829463
                    02         4303.780642
                    03         1074.314935
                    04         1092.766152
                    05          934.979390
...                                    ...
lowE(2)/Clear-VB75  08           25.472227
                    09           33.326059
                    10           36.690200
                    11           32.555067
                    12           28.279800

[600 rows x 1 columns]

In [36]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [37]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [38]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [39]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [40]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [41]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [42]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [43]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [44]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [45]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [46]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [47]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

allHourlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($)
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.215039
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.234931
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.251555
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267499
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136
...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.004310
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.053998
